In [1]:
%load_ext tensorboard
%tensorboard --logdir ./result

In [1]:
import math
import random
from typing import List, Optional, Tuple, Union
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
from torch import nn
import os
from torch.utils.data import IterableDataset, Dataset
import json
import numpy as np
from transformers import PreTrainedModel
from transformers.modeling_outputs import CausalLMOutputWithPast
from transformers import PretrainedConfig
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer, DefaultDataCollator, DataCollatorForTokenClassification, AutoConfig
from train_llama2_from_scratch import Config, LLM

/Users/yingyao/miniconda3/envs/transformer-practice/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
AutoConfig.register("custom_gpt", Config)
AutoModelForCausalLM.register(Config, LLM)

In [7]:
# eval result
query = "北京在哪个国家？"
query = "1+1等于几？"

reload_model = AutoModelForCausalLM.from_pretrained('/Users/yingyao/Desktop/Code/GetHandsDirty.nosync/sft/model').to(device)
tokenizer = AutoTokenizer.from_pretrained("./tokenizer", use_fast=True)
input_ids = [tokenizer.bos_token_id] + tokenizer.encode(query)
input_data = {'input_ids': torch.tensor(input_ids, device=device).unsqueeze(0), "labels":None} # unsqueeze(0) to insert a dim at index 0 for batch
input_data

{'input_ids': tensor([[   2,  731,   14,   20, 6239, 1919,  815]]),
 'labels': None}

In [8]:
tokenizer.decode(input_data['input_ids'][0])

'<s> 1+1等于几？'

In [9]:
for token in reload_model.generate(inputs=input_data, eos=tokenizer.eos_token_id, max_new_tokens=100, stream=False):
    print(tokenizer.decode(token[0]))

<|im_start|>好的，我会试试的。谢谢你的建议，祝你度过一个愉快的时光！<|im_end|> <|im_start|>提取一下文章中有关塑料污染的危害。全球变暖导致的一系列会导致大量污染，如二氧化碳、甲烷和氧化亚流失等。<|im_end|> <|im


In [12]:
reload_model = AutoModelForCausalLM.from_pretrained('/Users/yingyao/Desktop/Code/GetHandsDirty.nosync/sft/result/sft/checkpoint-6441').to(device)
input = f'<s>user\n{query}</s>\n<s>assistant\n'
input_ids = tokenizer.encode(input)
input_data = {'input_ids': torch.tensor(input_ids, device=device).unsqueeze(0), "labels":None} # unsqueeze(0) to insert a dim at index 0 for batch
for token in reload_model.generate(inputs=input_data, eos=tokenizer.eos_token_id, max_new_tokens=100, stream=False):
    print(tokenizer.decode(token[0]))

 因此，2只是两只鸟，只需要有2只鸟，它会唯一共有多少只鸟，但需要根据具体的身份来确定。

2 = 24只鸟

所以答案是：

从身到一只鸟是唱歌。


In [14]:
reload_model = AutoModelForCausalLM.from_pretrained('/Users/yingyao/Desktop/Code/GetHandsDirty.nosync/sft/result/dpo/checkpoint-10').to(device)
input = f'<s>user\n{query}</s>\n<s>assistant\n'
input_ids = tokenizer.encode(input)
input_data = {'input_ids': torch.tensor(input_ids, device=device).unsqueeze(0), "labels":None} # unsqueeze(0) to insert a dim at index 0 for batch
for token in reload_model.generate(inputs=input_data, eos=tokenizer.eos_token_id, max_new_tokens=100, stream=False):
    print(tokenizer.decode(token[0]))

 又<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
